# Selenium https://rollbit.com/ trade scrape

In [23]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install selenium
!{sys.executable} -m pip install webdriver-manager
!{sys.executable} -m pip install bs4
!{sys.executable} -m pip install pandas
# !{sys.executable} -m apt-get update 
# !{sys.executable} -m apt install chromium-chromedriver
!{sys.executable} -m pip install ipyparallel
!{sys.executable} -m pip install matplotlib

     ---------------------------------------- 7.2/7.2 MB 3.5 MB/s eta 0:00:00
     ---------------------------------------- 55.3/55.3 kB 3.0 MB/s eta 0:00:00
  Using cached cycler-0.11.0-py3-none-any.whl (6.4 kB)
     ---------------------------------------- 3.3/3.3 MB 3.6 MB/s eta 0:00:00
  Using cached fonttools-4.33.3-py3-none-any.whl (930 kB)


In [24]:
import time
import datetime
import pandas as pd
import ipyparallel as ipp
from datetime import timezone
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup


In [52]:
SLEEP = 0.5
ignored_exceptions = (StaleElementReferenceException,)
master_list = []

In [ ]:
# TEST
import gspread_dataframe as gd
import gspread as gs
gc = gs.service_account(filename="your/cred/file.json")

def export_to_sheets(worksheet_name,df,mode='r'):
    ws = gc.open("SHEET_NAME").worksheet("worksheet_name")
    if(mode=='w'):
        ws.clear()
        gd.set_with_dataframe(worksheet=ws,dataframe=df,include_index=False,include_column_header=True,resize=True)
        return True
    elif(mode=='a'):
        ws.add_rows(df.shape[0])
        gd.set_with_dataframe(worksheet=ws,dataframe=df,include_index=False,include_column_header=False,row=ws.row_count+1,resize=False)
        return True
    else:
        return gd.get_as_dataframe(worksheet=ws)
    
df = pd.DataFrame.from_records([{'a': i, 'b': i * 2} for i in range(100)])
export_to_sheets("SHEET_NAME",df,'a')

In [30]:
def btn_click(xpath):
    try:        
        driver.execute_script("arguments[0].click();", WebDriverWait(
            driver, 1, SLEEP, ignored_exceptions=ignored_exceptions).until(
            EC.presence_of_element_located((By.XPATH, xpath))))
    except Exception as e:
        print(e)

In [48]:
def update(html):
    _up = '#72f238'
    dt = datetime.datetime.now(timezone.utc)
    time = dt.strftime('%Y-%m-%d-%H:%M:%S.%f')[:-3]

    soup = BeautifulSoup(html, 'html.parser')
    trs = soup.find('tbody').find_all('tr')

    for tr in trs:
        data = {}
        t = tr.find_all('td')
        
        data['time']         = time
        data['player']       = t[0].text
        data['trade']        = t[1].find('a')['href'].split('/')[2]
        _color               = t[1].find('svg', {'class': 'css-1pzuxf4'})['color']
        if _color == _up:
            data['position'] = 'UP'
        else:
            data['position'] = 'DOWN'
        data['wager']        = t[2].find('div').text
        data['multiplier']   = t[3].find('div').text
        data['pnl']          = t[4].find('div').text
        data['roi']          = t[5].find('div').text        

        print(data)
        master_list.append(data)        
        break   

In [5]:
# SS5 = '74.208.76.181:1080'
options = Options()
chrome_options = webdriver.ChromeOptions()
# options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
# options.add_argument('--proxy-server=socks5://' + SS5)
# driver = webdriver.Chrome('chromedriver',chrome_options=options)
driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 102.0.5005
[WDM] - Get LATEST chromedriver version for 102.0.5005 google-chrome
[WDM] - Driver [C:\Users\Lux\.wdm\drivers\chromedriver\win32\102.0.5005.61\chromedriver.exe] found in cache
C:\Users\Lux\AppData\Local\Temp\ipykernel_5420\1860921914.py:9: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)
C:\Users\Lux\AppData\Local\Temp\ipykernel_5420\1860921914.py:9: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(ChromeDriverManager().install(), chrome_options=options)


In [6]:
driver.get('https://rollbit.com')
# html = driver.page_source
# html

In [49]:
_xpath_nav_trades = '//div[contains(text(),"Trades")]'
btn_click(_xpath_nav_trades)

In [58]:
_c = 0
_prev = ''
_now = ''
_xpath_table = '//table'

while 1:
    try:
        html_table = driver.find_element(By.XPATH, _xpath_table).get_attribute('innerHTML')
        _now = hash(html_table)
        if _prev != _now:
            _prev = _now
            update(html_table)
            df = pd.DataFrame(master_list)
            %store df
    except:
        print('Something went wrong, WebDown or InternetDown?')
    
    _c += 1
    if _c > 50000: 
        print('end')
        break
#     time.sleep(0.1)

{'time': '2022-05-29-23:31:16.283', 'player': 'dootmonster', 'trade': 'Rollercoaster', 'position': 'DOWN', 'wager': '0.14', 'multiplier': 'x100.00', 'pnl': '$0.03', 'roi': '21.43%'}
Stored 'df' (DataFrame)
{'time': '2022-05-29-23:31:30.809', 'player': 'dootmonster', 'trade': 'Rollercoaster', 'position': 'DOWN', 'wager': '0.17', 'multiplier': 'x200.00', 'pnl': '-$0.17', 'roi': '-100.00%'}
Stored 'df' (DataFrame)
{'time': '2022-05-29-23:31:31.747', 'player': 'shit trade ', 'trade': 'BTC', 'position': 'UP', 'wager': '20.00', 'multiplier': 'x1,000.00', 'pnl': '-$20.00', 'roi': '-100.00%'}
Stored 'df' (DataFrame)
{'time': '2022-05-29-23:31:51.800', 'player': 'shit trade ', 'trade': 'BTC', 'position': 'UP', 'wager': '20.00', 'multiplier': 'x1,000.00', 'pnl': '-$20.00', 'roi': '-100.00%'}
Stored 'df' (DataFrame)
{'time': '2022-05-29-23:32:21.754', 'player': 'Hidden', 'trade': 'TRX', 'position': 'UP', 'wager': '3.00', 'multiplier': 'x200.00', 'pnl': '$0.13', 'roi': '4.33%'}
Stored 'df' (DataFr

{'time': '2022-05-29-23:40:16.780', 'player': 'shyngo', 'trade': 'BTC', 'position': 'DOWN', 'wager': '5.00', 'multiplier': 'x1,000.00', 'pnl': '-$1.18', 'roi': '-23.60%'}
Stored 'df' (DataFrame)
{'time': '2022-05-29-23:40:24.746', 'player': 'Yurcio', 'trade': 'SOL', 'position': 'UP', 'wager': '58.00', 'multiplier': 'x100.00', 'pnl': '-$5.00', 'roi': '-8.62%'}
Stored 'df' (DataFrame)
{'time': '2022-05-29-23:40:32.256', 'player': 'JoeyShitCoin', 'trade': 'Rollercoaster', 'position': 'UP', 'wager': '0.16', 'multiplier': 'x500.00', 'pnl': '-$0.16', 'roi': '-100.00%'}
Stored 'df' (DataFrame)
{'time': '2022-05-29-23:40:33.249', 'player': 'Hidden', 'trade': 'Rollercoaster', 'position': 'DOWN', 'wager': '504.00', 'multiplier': 'x200.00', 'pnl': '$129.81', 'roi': '25.76%'}
Stored 'df' (DataFrame)
{'time': '2022-05-29-23:40:44.339', 'player': 'Hidden', 'trade': 'Rollercoaster', 'position': 'DOWN', 'wager': '635.00', 'multiplier': 'x200.00', 'pnl': '$150.12', 'roi': '23.64%'}
Stored 'df' (DataFra

In [55]:
df = pd.DataFrame(master_list)

In [56]:
df

,time,player,trade,position,wager,multiplier,pnl,roi
0,2022-05-29-01:55:47.128,E-saurus,Rollercoaster,UP,2.50,x100.00,$0.67,26.80%
1,2022-05-29-01:56:40.398,Hidden,Rollercoaster,DOWN,0.25,x600.00,-$0.25,-100.00%
2,2022-05-29-01:56:43.394,Hidden,Rollercoaster,DOWN,0.25,x600.00,-$0.25,-100.00%
3,2022-05-29-01:56:43.433,Hidden,Rollercoaster,DOWN,0.25,x600.00,-$0.25,-100.00%
4,2022-05-29-01:56:46.396,Hidden,Rollercoaster,DOWN,0.25,x600.00,-$0.25,-100.00%
...,...,...,...,...,...,...,...,...
123,2022-05-29-02:11:22.434,boardo,Rollercoaster,UP,40.00,x600.00,-$18.86,-47.15%
124,2022-05-29-02:11:23.906,boardo,Rollercoaster,UP,40.00,x600.00,-$18.86,-47.15%
125,2022-05-29-02:11:27.931,boardo,Rollercoaster,DOWN,40.00,x600.00,-$18.86,-47.15%
126,2022-05-29-02:11:29.930,boardo,Rollercoaster,DOWN,40.00,x600.00,$9.21,23.03%


In [ ]:
df.to_csv('./data/sample_rollbit.csv', index=False)

In [ ]:
driver.quit()

In [15]:
html_table = driver.find_element(By.XPATH, _xpath_table).get_attribute('innerHTML')
update(html_table)

{'time': '2022-05-29-01:10:25.082', 'player': 'boardo', 'trade': 'Rollercoaster', 'position': 'UP', 'wager': '40.00', 'multiplier': 'x600.00', 'pnl': '$39.96', 'roi': '99.90%'}


In [14]:
task_durations = [1] * 10
# request a cluster
with ipp.Cluster(n=4) as rc:
    # get a view on the cluster
    view = rc.load_balanced_view()
    # submit the tasks
    asyncresult = view.map_async(time.sleep, task_durations)
    # wait interactively for results
    asyncresult.wait_interactive()
    # retrieve actual results
    result = asyncresult.get()
# at this point, the cluster processes have been shutdown

Starting 4 engines with <class 'ipyparallel.cluster.launcher.LocalEngineSetLauncher'>
sleep: 100%|████████████████████████████████████████████████████████████████████████| 10/10 [00:03<00:00,  3.20tasks/s]
Stopping engine(s): 1653786613
engine set stopped 1653786613: {'engines': {'0': {'exit_code': 1, 'pid': 21928, 'identifier': '0'}, '1': {'exit_code': 1, 'pid': 17660, 'identifier': '1'}, '2': {'exit_code': 1, 'pid': 22072, 'identifier': '2'}, '3': {'exit_code': 1, 'pid': 2312, 'identifier': '3'}}, 'exit_code': 1}
Stopping controller
Controller stopped: {'exit_code': 1, 'pid': 25348, 'identifier': 'ipcontroller-1653786612-yjs2-5420'}
